<img src="PEST++V3_cover.jpeg" style="float: left">

<img src="flopylogo.png" style="float: right">

<img src="AW&H2015.png" style="float: center">

# Looking at Parameter Sensitivity

We have already discussed the Jacobian matrix in a few places. It is calculated by perturbing the parameter (usually 1%) and tracking what happens to each observation.  In a general form the sensitivity equation looks like eq. 9.7 Anderson et al. 2015:

<img src="Sensitivity_eq.png" style="float: center">

This is key for derivative-based parameter estimation because, as we've seen, this allows us to efficiently compute upgraded parameters to try during the lambda search.  But the Jacobian matrix can give us insight about the model in and of itself. 

Let's take a look at it more closely and see what we can learn from it and how to handle such information as the number of parameters rises.

In [ ]:
%matplotlib inline
import os
import sys
sys.path.append('..')
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyemu
from matplotlib.backends.backend_pdf import PdfPages
runall_flag = False
import sensitivity_identifiability_helper as sih

Bring in the model...

In [ ]:
import freyberg_setup as fs
fs.setup_pest_kr()
fs.setup_pest_pp()
working_dir = fs.WORKING_DIR_KR
pst_name = fs.PST_NAME_KR

## First read in the PST file and find what are the starting values for K and R

In [ ]:
inpst = pyemu.Pst(os.path.join(working_dir,pst_name))
inpst.parameter_data

### Now let's tell PEST++ to calculate the Jacobian matrix by changing NOPTMAX from 0 to -1 
(recall NOPTMAX=-1 calculates the Jacobian but not all the statistics, which is NOPTMAX=-2)

In [ ]:
inpst = pyemu.Pst(os.path.join(working_dir,pst_name))
inpst.control_data.noptmax =  -1
inpst.write(os.path.join(working_dir,pst_name.replace(".pst",".final.pst")))

### Now let's calculate the sensitivity by PEST++
(recall a Jacobian matrix takes a minimum of NPAR + 1, which is 4 runs for this case)

In [ ]:
 pyemu.helpers.run("pestpp {0}".format(pst_name.replace(".pst",".final.pst")),cwd=working_dir)

# We can also calculate one for a more complicated Pilot Points model

In [ ]:
inpst = pyemu.Pst(os.path.join(fs.WORKING_DIR_PP,fs.PST_NAME_PP))
inpst.control_data.noptmax =  -1
inpst.write(os.path.join(fs.WORKING_DIR_PP,'freyberg_pp_jac.pst'))

In [ ]:
if runall_flag is True:
    os.chdir(fs.WORKING_DIR_PP)
    pyemu.helpers.start_slaves('.', 'pestpp', 'freyberg_pp_jac.pst', num_slaves=15,master_dir='.')
    os.chdir('..')
else:
    if not os.path.exists(fs.WORKING_DIR_PP):
        os.mkdir(fs.WORKING_DIR_PP)
    shutil.copy2('freyberg_pp_jac.jcb',os.path.join(fs.WORKING_DIR_PP, 'freyberg_pp_jac.jcb'))

### Look at the Jacobian matrix---gradients of parameters wrt. observations

For each parameter-observation combination, we can see how much the observation value changes due to a small change in the parameter. If $y$ are the observations and $x$ are the parameters, the equation for the $i^th$ observation with respect to the $j^th$ parameter is:  
## $\frac{\partial y_i}{\partial x_j}$
This can be approximated by finite differences as :  
## $\frac{\partial y_i}{\partial x_j}~\frac{y\left(x+\Delta x \right)-y\left(x\right)}{\Delta x}$

### First we can read in a couple Jacobian matrices -- one from our simple model, and one from a more complex one

In [ ]:
jac_simple = pyemu.Jco.from_binary(os.path.join(working_dir,'freyberg_kr.final.jcb'))
jac_complex = pyemu.Jco.from_binary(os.path.join(fs.WORKING_DIR_PP, 'freyberg_pp_jac.jcb'))

### These are now matrices. How big are they?

In [ ]:
print ('simple  --> {0} rows x {1} columns'.format(*jac_simple.shape))
print ('complex --> {0} rows x {1} columns'.format(*jac_complex.shape))


In [ ]:
# Let's drop all the forecasts and regularization information
jac_simple.drop([x for x in jac_simple.df().index if x.startswith('pr')], axis=0)
jac_simple.drop([x for x in jac_simple.df().index if x.startswith('fr')], axis=0)
jac_simple.drop('travel_time', axis=0)

jac_complex.drop([x for x in jac_complex.df().index if x.startswith('pr')], axis=0)
jac_complex.drop([x for x in jac_complex.df().index if x.startswith('fr')], axis=0)
jac_complex.drop('travel_time', axis=0)

In [ ]:
jac_simple.shape

In [ ]:
sih.plot_Jacobian(jac_simple)

## how about just the first 20 observations?

In [ ]:
sih.plot_Jacobian(jac_simple[:20,:])

In [ ]:
sih.plot_Jacobian(jac_complex[:20,:])

# Can be more informative to look at sensitivity spatially

In [ ]:
print(jac_complex.row_names)

In [ ]:
sih.plot_jacobian_spatial(jac_complex,'cr34c08_19700102');

In [ ]:
with PdfPages('allsens.pdf') as ofp:
    for cob in jac_complex.row_names:
        if 'flx' not in cob and 'vol' not in cob:
            cf = sih.plot_jacobian_spatial(jac_complex, cob)
            ofp.savefig()
            plt.close('all')

# How about Composite Scaled Sensitivities
In the traditional, overdetermined regression world, CSS was a popular metric. CSS is Composite Scaled Sensitivitity.

In Hill and Tiedeman (2007) this is calculated as: 
## ${css_{j}=\sqrt{\left(\sum_{i-1}^{ND}\left(\frac{\partial y'_{i}}{\partial b_{j}}\right)\left|b_{j}\right|\sqrt{w_{ii}}\right)/ND}}$

In PEST, Doherty calculates it slightly differently in that scaling by the parameter values happens automatically when the parameter is subjected to a log-transform. This is due to a correction that must be made in calculating the Jacobian matrix and follows from the chain rule of derivatives.


In [ ]:
la = pyemu.LinearAnalysis(jco=os.path.join(os.path.join(fs.WORKING_DIR_PP, 'freyberg_pp_jac.jcb')))

In [ ]:
plt.figure(figsize=(8,4))
ax = la.get_par_css_dataframe()['pest_css'].sort_values(ascending=False).plot(kind='bar')
ax.set_yscale('log')

### Now let's consider correlation and posterior covariance

In [ ]:
sc = pyemu.Schur(os.path.join(os.path.join(fs.WORKING_DIR_PP, 'freyberg_pp_jac.jcb')))
covar = pyemu.Cov(sc.xtqx.x, names=sc.xtqx.row_names)
covar.df().head()

In [ ]:
R = covar.to_pearson()
plt.imshow(R.df(), interpolation='nearest', cmap='viridis')
plt.colorbar()

In [ ]:
cpar = 'hk10'
R.df().loc[cpar][np.abs(R.df().loc[cpar])>.5]

In [ ]:
R_plot = R.df().as_matrix()
R_plot[np.abs(R_plot)>.9] = np.nan
plt.imshow(R_plot, interpolation='nearest', cmap='viridis')
plt.colorbar()